- Import packages to run SQL in Jupyter Notebooks.
- Upload csv files to create a database on local machine.

In [2]:
import pandas as pd
import sqlalchemy

In [3]:
engine = sqlalchemy.create_engine('sqlite:////Users/harley/DA_Projects/Project_Databases/EyeglassStore.db')

In [4]:
data = pd.read_csv('/Users/harley/DA_Projects/Datasets/Eyeglass_Store/survey.csv')

In [5]:
data.to_sql(con=engine,name='survey')

In [6]:
data = pd.read_csv('/Users/harley/DA_Projects/Datasets/Eyeglass_Store/quiz.csv')

In [7]:
data.to_sql(con=engine,name='quiz')

In [8]:
data = pd.read_csv('/Users/harley/DA_Projects/Datasets/Eyeglass_Store/home_try_on.csv')

In [9]:
data.to_sql(con=engine,name='home_try_on')

In [10]:
data = pd.read_csv('/Users/harley/DA_Projects/Datasets/Eyeglass_Store/purchase.csv')

In [11]:
data.to_sql(con=engine,name='purchase')

- Load extension to run SQL queries.
- Open Eyeglass Store database. 

In [12]:
%load_ext sql

In [13]:
%sql sqlite:////Users/harley/DA_Projects/Project_Databases/EyeglassStore.db

- Analysis of Style Quiz Funnel to calculate conversion rates (cvr). 
- This query calculates the percentage of users who answered each question in the style quiz.
    - Q5 (When was your last eye exam?) had the lowest completion rate at 54%. 
    - This is most likely because users do not remember the date of their last eye exam off hand. 
    - To increase the user response rate, questions should pertain to personal preferences.

In [19]:
%%sql

SELECT question,
    COUNT(*) AS 'num_responses',
    100.0 * COUNT(*) / 500.0 AS 'cvr'
FROM survey
GROUP BY 1

 * sqlite:////Users/harley/DA_Projects/Project_Databases/EyeglassStore.db
Done.


question,num_responses,cvr
1. What are you looking for?,500,100.0
2. What's your fit?,475,95.0
3. How old are you?,380,76.0
4. Which colors do you like?,361,72.2
5. When was your last eye exam?,270,54.0


- Creation of new table where each record represents a single user and the following attributes: 
    - Home try-on completion (TRUE/FALSE)
    - Number of pairs received (3 pairs/5 pairs)
    - Purchase completion (TRUE/FALSE)

In [25]:
%%sql

SELECT DISTINCT q.user_id,
    h.user_id IS NOT NULL AS 'is_home_try_on',
    h.number_of_pairs AS 'num_pairs',
    p.user_id IS NOT NULL AS 'is_purchase'
FROM quiz q
LEFT JOIN home_try_on h
    ON q.user_id = h.user_id
LEFT JOIN purchase p
    ON q.user_id = p.user_id
LIMIT 5

 * sqlite:////Users/harley/DA_Projects/Project_Databases/EyeglassStore.db
Done.


user_id,is_home_try_on,num_pairs,is_purchase
4e8118dc-bb3d-49bf-85fc-cca8d83232ac,1,3 pairs,0
291f1cca-e507-48be-b063-002b14906468,1,3 pairs,1
75122300-0736-4087-b6d8-c0c5373a1a04,0,None,0
75bc6ebd-40cd-4e1d-a301-27ddd93b12e2,1,5 pairs,0
ce965c4d-7a2b-4db6-9847-601747fa7812,1,3 pairs,1


- A/B test using above table as a temporary table to determine whether receiving 3 pairs or 5 pairs of glasses is more likely to result in a purchase.
    - 59.39% of users who received 5 pairs made a purchase.
    - 40.61% of users who received 3 pairs made a purchase.
    - This shows a positive correlation between receiving more pairs of glasses and making a purchase.

In [21]:
%%sql

WITH funnel AS (
    SELECT DISTINCT q.user_id,
    h.user_id IS NOT NULL AS 'is_home_try_on',
    h.number_of_pairs,
    p.user_id IS NOT NULL AS 'is_purchase'
FROM quiz q
LEFT JOIN home_try_on h
    ON q.user_id = h.user_id
LEFT JOIN purchase p
    ON q.user_id = p.user_id
)
SELECT COUNT(DISTINCT CASE
        WHEN number_of_pairs = '3 pairs' AND is_purchase = 1 THEN user_id END) AS '3_pair_purchase',
    ROUND((201.0 / 495.0) * 100.0, 2) AS '3_pair_purchase_%',
    COUNT(DISTINCT CASE
         WHEN number_of_pairs = '5 pairs' AND is_purchase = 1 THEN user_id END) AS '5_pair_purchase',
    ROUND((294.0 / 495.0) * 100.0, 2) AS '5_pair_purchase_%'
FROM funnel

 * sqlite:////Users/harley/DA_Projects/Project_Databases/EyeglassStore.db
Done.


3_pair_purchase,3_pair_purchase_%,5_pair_purchase,5_pair_purchase_%
201,40.61,294,59.39


- Analysis of Home Try-On Funnel to calculate conversion rates. 
    - Out of 1000 initial users who took the style quiz, 750 completed the home try-on, and 495 made a purchase.
    - 75% of users advanced from style quiz phase to home try-on phase.
    - 66% of users advanced from home try-on phase to purchase phase. 

In [22]:
%%sql

WITH funnel AS (
    SELECT DISTINCT q.user_id,
    h.user_id IS NOT NULL AS 'is_home_try_on',
    h.number_of_pairs,
    p.user_id IS NOT NULL AS 'is_purchase'
FROM quiz q
LEFT JOIN home_try_on h
    ON q.user_id = h.user_id
LEFT JOIN purchase p
    ON q.user_id = p.user_id
)
SELECT COUNT(*) AS 'num_quiz',
    SUM(is_home_try_on) AS 'num_try_on',
    SUM(is_purchase) AS 'num_purchase',
    100.0 * SUM(is_home_try_on) / COUNT(*) AS 'quiz_to_try_on_cvr',
    100.0 * SUM(is_purchase) / SUM(is_home_try_on) AS 'try_on_to_purchase_cvr'
FROM funnel

 * sqlite:////Users/harley/DA_Projects/Project_Databases/EyeglassStore.db
Done.


num_quiz,num_try_on,num_purchase,quiz_to_try_on_cvr,try_on_to_purchase_cvr
1000,750,495,75.0,66.0
